# JEM092 Asset Pricing
# Seminar 11
## Lukáš Petrásek
### Charles University
### lukas.petrasek@fsv.cuni.cz
## 28.4.2022

In [2]:
# Required packages
packages <- c("lubridate", "quantmod", "sandwich", "xts")

# Install packages not yet installed
installed_packages <- packages %in% rownames(installed.packages())
if (any(installed_packages == FALSE)) {
  install.packages(packages[!installed_packages])
  print(paste("Installing package ", packages[!installed_packages],"...", sep = ""))
}

# Packages loading
invisible(lapply(packages, library, character.only = TRUE))
rm(list = ls()) #Clean environment
print('All packages loaded successfully...')

# Miscellaneous
options(repr.plot.width = 6, repr.plot.height = 5)
set.seed(420)

[1] "All packages loaded successfully..."


# Data preparation

In [3]:
load('Asset_Pricing_seminar_data.RData')
ls() 

Warning message in readChar(con, 5L, useBytes = TRUE):
"cannot open compressed file 'Asset_Pricing_seminar_data.RData', probable reason 'No such file or directory'"

ERROR: Error in readChar(con, 5L, useBytes = TRUE): cannot open the connection


In [ ]:
START_DATE <- '2005-01-01'
END_DATE <- '2021-03-31'

# Load stock specific data.
prices <- c()
bm_ratios <- c()
market_caps <- c()
for (i in 1:length(OHLCV_sap100)) {
    ticker <- as.character(names(OHLCV_sap100)[i])
    stock_data <- getSymbols(
        ticker,
        from = START_DATE,
        to = END_DATE,
        src = 'yahoo',
        warnings = FALSE,
        auto.assign = FALSE
    )

    stock_prices = stock_data[, 6]
    stock_bm_ratios <- book_value_sap100[[i]][, 2] / book_value_sap100[[i]][, 1]
    stock_market_caps <- MktCap_sap100[[i]]
    colnames(stock_prices) <- ticker
    colnames(stock_bm_ratios) <- ticker
    colnames(stock_market_caps) <- ticker
    prices <- cbind(prices, stock_prices)
    bm_ratios <- cbind(bm_ratios, stock_bm_ratios)
    market_caps <- cbind(market_caps, stock_market_caps)
}

# Forward-fill book-to-market ratios (observed quarterly) and drop the last row.
bm_ratios <- na.locf(bm_ratios)
bm_ratios <- head(bm_ratios, -1)

# Load daily FF3 factors.
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_TXT.zip",
    destfile = "F-F_Research_Data_Factors_daily.zip"
)
unzip("F-F_Research_Data_Factors_daily.zip")
ff3_factors <- read.delim(
    'F-F_Research_Data_Factors_daily.txt',
    col.names = c('t', 'mkt_rf', 'smb', 'hml', 'rf'),
    sep = '',
    nrows = 24957,
    header = FALSE,
    skip = 5,
    stringsAsFactors = FALSE
)
ff3_factors[['t']] <- as.Date(as.character(ff3_factors[['t']]), '%Y%m%d')
ff3_factors <- as.xts(ff3_factors[, 2:5], order.by = ff3_factors[['t']])
ff3_factors <- ff3_factors / 100

# Load monthly FF3 factors.
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_TXT.zip",
    destfile = "F-F_Research_Data_Factors.zip"
)
unzip("F-F_Research_Data_Factors.zip")
monthly_ff3_factors <- read.delim(
    'F-F_Research_Data_Factors.txt',
    col.names = c('t', 'mkt.rf', 'smb', 'hml', 'rf'),
    sep = '',
    nrows = 1137,
    header = FALSE,
    skip = 4,
    stringsAsFactors = FALSE
)
monthly_ff3_factors[['t']] <- as.Date(paste0(as.character(monthly_ff3_factors[['t']]), '01'), '%Y%m%d')
monthly_ff3_factors <- as.xts(monthly_ff3_factors[, 2:5], order.by = monthly_ff3_factors[['t']])
monthly_ff3_factors <- monthly_ff3_factors / 100
index(monthly_ff3_factors) <- as.Date(as.yearmon(index(monthly_ff3_factors)), frac = 1)

In [ ]:
prices[1:5, 1:3]
market_caps[201:205, 1:3]
bm_ratios[20:25, 1:3]
ff3_factors[1:5,]
monthly_ff3_factors[1:5,]

In [ ]:
# Compute daily and monthly excess returns.
daily_returns <- lapply(prices, dailyReturn, USE.NAMES = TRUE)
monthly_returns <- lapply(prices, monthlyReturn, USE.NAMES = TRUE)
daily_returns <- do.call('cbind', daily_returns)
monthly_returns <- do.call('cbind', monthly_returns)
colnames(daily_returns) <- colnames(prices)
colnames(monthly_returns) <- colnames(prices)
index(monthly_returns) <- as.Date(as.yearmon(index(monthly_returns)), frac = 1)
for (i in 1:ncol(monthly_returns)) {
    daily_returns[, i] <- daily_returns[, i] - ff3_factors[, 4]
    monthly_returns[, i] <- monthly_returns[, i] - monthly_ff3_factors[, 4]
}

# Comupte monthly market caps and sizes.
sizes <- log(market_caps)
monthly_market_caps <- apply.monthly(market_caps, tail, 1)
monthly_sizes <- apply.monthly(sizes, tail, 1)
index(monthly_market_caps) <- as.Date(as.yearmon(index(monthly_market_caps)), frac = 1)
index(monthly_sizes) <- as.Date(as.yearmon(index(monthly_sizes)), frac = 1)
# Replace -Inf and Inf in `monthly_sizes` with NAs.
monthly_sizes <- as.xts(apply(monthly_sizes, 2, function(x) ifelse(is.finite(x), x, NA)))

In [ ]:
daily_returns[1:5, 1:3]
monthly_returns[1:5, 1:3]
monthly_market_caps[1:5, 1:3]
monthly_sizes[1:5, 1:3]

## Compute book-to-market ratios

In [ ]:
# Initialize an xts object for storing book-to-market ratios.
monthly_bm_ratios <- as.xts(
    matrix(
        nrow = nrow(bm_ratios),
        ncol = ncol(bm_ratios)
    ),
    order.by = index(bm_ratios)
)
names(monthly_bm_ratios) <- names(bm_ratios)

start_year <- as.numeric(format(min(index(monthly_bm_ratios)), '%Y'))
end_year <- as.numeric(format(max(index(monthly_bm_ratios)), '%Y'))

# Iterate over rows and compute BM ratios.
for (i in (start_year + 2):end_year) {
    start_day <- as.Date(paste0(i - 1, '06', '30'), format = '%Y%m%d')
    end_day <- as.Date(paste0(i, '05', '31'), format = '%Y%m%d')
    dates <- paste0(start_day, '/', end_day)
    end_of_previous_year <- as.Date(paste0(i - 2, '12', '31'), format = '%Y%m%d')
    for (j in 1:ncol(bm_ratios)) {
        monthly_bm_ratios[dates, j] <- as.numeric(bm_ratios[end_of_previous_year, j][[1]])
    }
}

In [ ]:
monthly_bm_ratios[41:55, 1:3]
bm_ratios[41:55, 1:3]

## Compute momentum

In [ ]:
# Initialize an xts object for storing momentum.
monthly_momentum <- as.xts(
    matrix(
        nrow = nrow(monthly_returns),
        ncol = ncol(monthly_returns)
    ),
    order.by = index(monthly_returns)
)
names(monthly_momentum) <- names(monthly_returns)

# Iterate over rows and compute momentum.
for (i in 2:nrow(monthly_momentum)) {
    current_day <- index(monthly_momentum[i])
    end_day <- index(monthly_momentum[i - 1])
    start_day <- end_day %m-% months(10)
    dates <- paste0(start_day, '/', end_day)
    returns <- monthly_returns[dates]

    if (nrow(returns) < 11) {
        next
    }

    for (j in 1:ncol(monthly_momentum)) {
        monthly_momentum[current_day, j] <- 100 * (prod(returns[, j] + 1) - 1)
    }
}

In [ ]:
monthly_momentum[11:15, 1:3]

## Compute short-term reversal

In [ ]:
# Initialize an xts object for storing reversal
monthly_reversal <- as.xts(
    matrix(
        nrow = nrow(monthly_returns),
        ncol = ncol(monthly_returns)
    ),
    order.by = index(monthly_returns)
)
names(monthly_reversal) <- names(monthly_returns)

# Iterate over rows and compute reversal.
for (i in 1:nrow(monthly_reversal)) {
    monthly_reversal[i,] <- monthly_returns[i] * 100
}

In [ ]:
monthly_reversal[11:15, 1:3]

## Estimate betas

Use the following regression equation:

$r_{i,t} = \alpha_i + \beta_i MKT_t + \epsilon_{i,t}$

- $i$ denotes stocks
- $t$ denotes time
- $r$ is the excess stock return
- $MKT$ is the excess market return
- $\alpha$ is the intercept
- $\beta$ is the slope coefficient
- $\epsilon$ is the error term

Let's illustrate the estimation on daily data for 5 different periods: 1, 3, 6, 12, and 24 months.

In [ ]:
# Initialize an xts object for storing betas.
betas_period <- as.xts(
    matrix(
        nrow = nrow(monthly_returns),
        ncol = ncol(monthly_returns)
    ),
    order.by = index(monthly_returns)
)
names(betas_period) <- names(monthly_returns)
betas <- list(betas_period, betas_period, betas_period, betas_period, betas_period)

# Iterate over rows and compute betas.
for (i in 1:nrow(betas_period)) {
    end_day <- index(betas_period[i])

    for (p_b in list(c(1, 1), c(3, 2), c(6, 3), c(12, 4), c(24, 5))) {
        start_day <- end_day %m-% months(p_b[[1]])
        dates <- paste0(start_day, '/', end_day)
        returns <- daily_returns[dates]
        market_returns <- ff3_factors[dates]

        # Estimate betas for each stock.
        for (j in 1:ncol(betas_period)) {
            stock_returns <- returns[, j]
            # Merge excess returns and market excess returns.
            model_data <- cbind(stock_returns, market_returns[, 1])
            model_data <- na.omit(model_data)
            # Don't estimate betas if the number of observations is lower than 200.
            if (nrow(model_data) < p_b[[1]] * 15) {
                next
            }
            model_betas <- lm(model_data[, 1] ~ model_data[, 2])
            # Save the given beta to `betas`.
            betas[[p_b[[2]]]][i, j] <- model_betas$coefficients[[2]]
        }
    }
}

monthly_betas_1m <- betas[[1]]
monthly_betas_3m <- betas[[2]]
monthly_betas_6m <- betas[[3]]
monthly_betas_12m <- betas[[4]]
monthly_betas_24m <- betas[[5]]

In [ ]:
monthly_betas_1m[1:5, 1:3]
monthly_betas_3m[1:5, 1:3]
monthly_betas_6m[11:15, 1:3]
monthly_betas_12m[21:25, 1:3]
monthly_betas_24m[21:25, 1:3]

# Value

## Portfolio analysis using monthly univariate decile portfolios

In [ ]:
n_portfolios <- 10
diff_portfolio <- paste(n_portfolios, '- 1')

# Initialize an xts object for storing portfolio returns.
returns <- as.xts(
    matrix(
        nrow = nrow(monthly_bm_ratios) - 1,
        ncol = n_portfolios + 1
    ),
    order.by = index(monthly_bm_ratios[2:nrow(monthly_bm_ratios)])
)
names(returns) <- c(1:n_portfolios, diff_portfolio)

# Iterate over rows, find breakpoints and compute monthly returns within the given value breakpoints.
for (i in 1:nrow(returns)) {
    current_month <- index(returns[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the month used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_bm_ratios <- monthly_bm_ratios[current_month]
    month_market_caps <- monthly_market_caps[next_month]
    # Replace NA market caps with 0s so that such observations have no weights.
    month_market_caps[is.na(month_market_caps)] <- 0

    breakpoints <- quantile(month_bm_ratios, 0:n_portfolios/n_portfolios, na.rm = TRUE)
    not_na <- !is.na(month_bm_ratios)

    for (j in 1:n_portfolios) {
        filter <- (breakpoints[[j]] < month_bm_ratios) & (month_bm_ratios < breakpoints[[j + 1]]) & not_na
        # Compute weighted average portfolio returns.
        returns[i, j] <- weighted.mean(t(month_returns[, filter]), t(month_market_caps[, filter]))
    }
    returns[i, diff_portfolio] <- returns[i, n_portfolios] - returns[i, 1]
}

# Compute overall average returns within portfolios and their standard errors.
results_bm_ratios <- as.data.frame(matrix(nrow = 2, ncol = n_portfolios + 1))
names(results_bm_ratios) <- c(1:n_portfolios, diff_portfolio)

for (i in 1:ncol(results_bm_ratios)) {
    model <- lm(na.omit(returns[, i]) ~ 1)
    results_bm_ratios[1, i] <- model$coefficients[[1]]
    results_bm_ratios[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
}

results_bm_ratios

## Fama-MacBeth regression analysis

In [ ]:
# Initialize an xts object for storing regression results.
fm_results <- as.xts(
    matrix(
        nrow = nrow(monthly_bm_ratios) - 1,
        ncol = 5
    ),
    order.by = index(monthly_bm_ratios[2:nrow(monthly_bm_ratios)])
)
names(fm_results) <- c('Intercept', 'Value', 'R^2', 'Adjusted R^2', 'N')

# Iterate over rows and perform cross-sectional regressions.
for (i in 1:nrow(fm_results)) {
    current_month <- index(fm_results[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_bm_ratios <- monthly_bm_ratios[current_month]

    if (ncol(month_bm_ratios) == sum(is.na(month_bm_ratios))) {
        next
    }

    # Save regression coefficients and other statistics.
    model <- lm(t(month_returns) ~ 1 + t(month_bm_ratios))
    fm_results[i, 1] <- model$coefficients[[1]]
    fm_results[i, 2] <- model$coefficients[[2]]
    fm_results[i, 3] <- summary(model)$r.squared
    fm_results[i, 4] <- summary(model)$adj.r.squared
    fm_results[i, 5] <- nobs(model)
}

# Compute time series means, standard errors.
final_fm_results <- as.data.frame(matrix(nrow = 2, ncol = 2))
names(final_fm_results) <- c('Intercept', 'Value')

for (i in 1:ncol(final_fm_results)) {
    model <- lm(na.omit(fm_results[, i]) ~ 1)
    final_fm_results[1, i] <- model$coefficients[[1]]
    final_fm_results[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
}

final_fm_results

## Fama-MacBeth regressions of returns on BM ratios, betas and sizes

In [ ]:
# Initialize an xts object for storing regression results.
fm_results <- as.xts(
    matrix(
        nrow = nrow(monthly_betas_24m) - 1,
        ncol = 7
    ),
    order.by = index(monthly_betas_24m[2:nrow(monthly_betas_24m)])
)
names(fm_results) <- c('Intercept', 'Value', 'Beta', 'Size', 'R^2', 'Adjusted R^2', 'N')

# Iterate over rows and perform cross-sectional regressions.
for (i in 1:nrow(fm_results)) {
    current_month <- index(fm_results[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_bm_ratios <- monthly_bm_ratios[current_month]
    month_betas <- monthly_betas_24m[current_month]
    month_sizes <- monthly_sizes[current_month]

    if (ncol(month_bm_ratios) == sum(is.na(month_bm_ratios))) {
        next
    }
    if (ncol(month_betas) == sum(is.na(month_betas))) {
        next
    }

    # Save regression coefficients and other statistics.
    model <- lm(t(month_returns) ~ 1 + t(month_bm_ratios) + t(month_betas) + t(month_sizes))
    fm_results[i, 1] <- model$coefficients[[1]]
    fm_results[i, 2] <- model$coefficients[[2]]
    fm_results[i, 3] <- model$coefficients[[3]]
    fm_results[i, 4] <- model$coefficients[[4]]
    fm_results[i, 5] <- summary(model)$r.squared
    fm_results[i, 6] <- summary(model)$adj.r.squared
    fm_results[i, 7] <- nobs(model)
}

# Compute time series means, standard errors.
final_fm_results <- as.data.frame(matrix(nrow = 2, ncol = 4))
names(final_fm_results) <- c('Intercept', 'Value', 'Beta', 'Size')

for (i in 1:ncol(final_fm_results)) {
    model <- lm(na.omit(fm_results[, i]) ~ 1)
    final_fm_results[1, i] <- model$coefficients[[1]]
    final_fm_results[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
}

final_fm_results

# Momentum

## Portfolio analysis using monthly univariate decile portfolios

In [ ]:
n_portfolios <- 10
diff_portfolio <- paste(n_portfolios, '- 1')

# Initialize an xts object for storing portfolio returns.
returns <- as.xts(
    matrix(
        nrow = nrow(monthly_momentum) - 1,
        ncol = n_portfolios + 1
    ),
    order.by = index(monthly_momentum[2:nrow(monthly_momentum)])
)
names(returns) <- c(1:n_portfolios, diff_portfolio)

# Iterate over rows, find breakpoints and compute monthly returns within the given momentum breakpoints.
for (i in 1:nrow(returns)) {
    current_month <- index(returns[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the month used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_momentum <- monthly_momentum[current_month]
    month_market_caps <- monthly_market_caps[next_month]
    # Replace NA market caps with 0s so that such observations have no weights.
    month_market_caps[is.na(month_market_caps)] <- 0

    breakpoints <- quantile(month_momentum, 0:n_portfolios/n_portfolios, na.rm = TRUE)
    not_na <- !is.na(month_momentum)

    for (j in 1:n_portfolios) {
        filter <- (breakpoints[[j]] < month_momentum) & (month_momentum < breakpoints[[j + 1]]) & not_na
        # Compute weighted average portfolio returns.
        returns[i, j] <- weighted.mean(t(month_returns[, filter]), t(month_market_caps[, filter]))
    }
    returns[i, diff_portfolio] <- returns[i, n_portfolios] - returns[i, 1]
}

# Compute overall average returns within portfolios and their standard errors.
results_momentum <- as.data.frame(matrix(nrow = 2, ncol = n_portfolios + 1))
names(results_momentum) <- c(1:n_portfolios, diff_portfolio)

for (i in 1:ncol(results_momentum)) {
    model <- lm(na.omit(returns[, i]) ~ 1)
    results_momentum[1, i] <- model$coefficients[[1]]
    results_momentum[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
}

results_momentum

## Fama-MacBeth regression analysis

In [ ]:
# Initialize an xts object for storing regression results.
fm_results <- as.xts(
    matrix(
        nrow = nrow(monthly_momentum) - 1,
        ncol = 5
    ),
    order.by = index(monthly_momentum[2:nrow(monthly_momentum)])
)
names(fm_results) <- c('Intercept', 'Momentum', 'R^2', 'Adjusted R^2', 'N')

# Iterate over rows and perform cross-sectional regressions.
for (i in 1:nrow(fm_results)) {
    current_month <- index(fm_results[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_momentum <- monthly_momentum[current_month]

    if (ncol(month_momentum) == sum(is.na(month_momentum))) {
        next
    }

    # Save regression coefficients and other statistics.
    model <- lm(t(month_returns) ~ 1 + t(month_momentum))
    fm_results[i, 1] <- model$coefficients[[1]]
    fm_results[i, 2] <- model$coefficients[[2]]
    fm_results[i, 3] <- summary(model)$r.squared
    fm_results[i, 4] <- summary(model)$adj.r.squared
    fm_results[i, 5] <- nobs(model)
}

# Compute time series means, standard errors.
final_fm_results <- as.data.frame(matrix(nrow = 2, ncol = 2))
names(final_fm_results) <- c('Intercept', 'Momentum')

for (i in 1:ncol(final_fm_results)) {
    model <- lm(na.omit(fm_results[, i]) ~ 1)
    final_fm_results[1, i] <- model$coefficients[[1]]
    final_fm_results[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
}

final_fm_results

## Fama-MacBeth regressions of returns on momentum, BM ratios, betas and sizes

In [ ]:
# Initialize an xts object for storing regression results.
fm_results <- as.xts(
    matrix(
        nrow = nrow(monthly_betas_24m) - 1,
        ncol = 8
    ),
    order.by = index(monthly_betas_24m[2:nrow(monthly_betas_24m)])
)
names(fm_results) <- c('Intercept', 'Momentum', 'Value', 'Beta', 'Size', 'R^2', 'Adjusted R^2', 'N')

# Iterate over rows and perform cross-sectional regressions.
for (i in 1:nrow(fm_results)) {
    current_month <- index(fm_results[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_momentum <- monthly_momentum[current_month]
    month_bm_ratios <- monthly_bm_ratios[current_month]
    month_betas <- monthly_betas_24m[current_month]
    month_sizes <- monthly_sizes[current_month]

    if (ncol(month_bm_ratios) == sum(is.na(month_bm_ratios))) {
        next
    }
    if (ncol(month_betas) == sum(is.na(month_betas))) {
        next
    }

    # Save regression coefficients and other statistics.
    model <- lm(t(month_returns) ~ 1 + t(month_momentum) + t(month_bm_ratios) + t(month_betas) + t(month_sizes))
    fm_results[i, 1] <- model$coefficients[[1]]
    fm_results[i, 2] <- model$coefficients[[2]]
    fm_results[i, 3] <- model$coefficients[[3]]
    fm_results[i, 4] <- model$coefficients[[4]]
    fm_results[i, 5] <- model$coefficients[[5]]
    fm_results[i, 6] <- summary(model)$r.squared
    fm_results[i, 7] <- summary(model)$adj.r.squared
    fm_results[i, 8] <- nobs(model)
}

# Compute time series means, standard errors.
final_fm_results <- as.data.frame(matrix(nrow = 2, ncol = 5))
names(final_fm_results) <- c('Intercept', 'Momentum', 'Value', 'Beta', 'Size')

for (i in 1:ncol(final_fm_results)) {
    model <- lm(na.omit(fm_results[, i]) ~ 1)
    final_fm_results[1, i] <- model$coefficients[[1]]
    final_fm_results[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
}

final_fm_results

# Short-term Reversal

## Portfolio analysis using monthly univariate decile portfolios

### Equally weighted returns

In [ ]:
n_portfolios <- 10
diff_portfolio <- paste(n_portfolios, '- 1')

# Initialize an xts object for storing portfolio returns.
returns <- as.xts(
    matrix(
        nrow = nrow(monthly_reversal) - 1,
        ncol = n_portfolios + 1
    ),
    order.by = index(monthly_reversal[2:nrow(monthly_reversal)])
)
names(returns) <- c(1:n_portfolios, diff_portfolio)

# Iterate over rows, find breakpoints and compute monthly returns within the given reverfsal breakpoints.
for (i in 1:nrow(returns)) {
    current_month <- index(returns[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the month used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_reversal <- monthly_reversal[current_month]

    breakpoints <- quantile(month_reversal, 0:n_portfolios/n_portfolios, na.rm = TRUE)
    not_na <- !is.na(month_reversal)

    for (j in 1:n_portfolios) {
        filter <- (breakpoints[[j]] < month_reversal) & (month_reversal < breakpoints[[j + 1]]) & not_na
        # Compute equal-weighted average portfolio returns.
        returns[i, j] <- mean(t(month_returns[, filter]))
    }
    returns[i, diff_portfolio] <- returns[i, n_portfolios] - returns[i, 1]
}

# Compute overall average returns within portfolios and their standard errors.
results_reversal <- as.data.frame(matrix(nrow = 2, ncol = n_portfolios + 1))
names(results_reversal) <- c(1:n_portfolios, diff_portfolio)

for (i in 1:ncol(results_reversal)) {
    model <- lm(na.omit(returns[, i]) ~ 1)
    results_reversal[1, i] <- model$coefficients[[1]]
    results_reversal[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
}

results_reversal

### Weighted returns

In [ ]:
n_portfolios <- 10
diff_portfolio <- paste(n_portfolios, '- 1')

# Initialize an xts object for storing portfolio returns.
returns <- as.xts(
    matrix(
        nrow = nrow(monthly_reversal) - 1,
        ncol = n_portfolios + 1
    ),
    order.by = index(monthly_reversal[2:nrow(monthly_reversal)])
)
names(returns) <- c(1:n_portfolios, diff_portfolio)

# Iterate over rows, find breakpoints and compute monthly returns within the given reverfsal breakpoints.
for (i in 1:nrow(returns)) {
    current_month <- index(returns[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the month used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_reversal <- monthly_reversal[current_month]
    month_market_caps <- monthly_market_caps[next_month]
    # Replace NA market caps with 0s so that such observations have no weights.
    month_market_caps[is.na(month_market_caps)] <- 0

    breakpoints <- quantile(month_reversal, 0:n_portfolios/n_portfolios, na.rm = TRUE)
    not_na <- !is.na(month_reversal)

    for (j in 1:n_portfolios) {
        filter <- (breakpoints[[j]] < month_reversal) & (month_reversal < breakpoints[[j + 1]]) & not_na
        # Compute weighted average portfolio returns.
        returns[i, j] <- weighted.mean(t(month_returns[, filter]), t(month_market_caps[, filter]))
    }
    returns[i, diff_portfolio] <- returns[i, n_portfolios] - returns[i, 1]
}

# Compute overall average returns within portfolios and their standard errors.
results_reversal <- as.data.frame(matrix(nrow = 2, ncol = n_portfolios + 1))
names(results_reversal) <- c(1:n_portfolios, diff_portfolio)

for (i in 1:ncol(results_reversal)) {
    model <- lm(na.omit(returns[, i]) ~ 1)
    results_reversal[1, i] <- model$coefficients[[1]]
    results_reversal[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
}

results_reversal

## Fama-MacBeth regressions of returns on reversal, momentum, BM ratios, betas and sizes

In [ ]:
# Initialize an xts object for storing regression results.
fm_results <- as.xts(
    matrix(
        nrow = nrow(monthly_betas_24m) - 1,
        ncol = 9
    ),
    order.by = index(monthly_betas_24m[2:nrow(monthly_betas_24m)])
)
names(fm_results) <- c('Intercept', 'Reversal', 'Momentum', 'Value', 'Beta', 'Size', 'R^2', 'Adjusted R^2', 'N')

# Iterate over rows and perform cross-sectional regressions.
for (i in 1:nrow(fm_results)) {
    current_month <- index(fm_results[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_reversal <- monthly_reversal[current_month]
    month_momentum <- monthly_momentum[current_month]
    month_bm_ratios <- monthly_bm_ratios[current_month]
    month_betas <- monthly_betas_24m[current_month]
    month_sizes <- monthly_sizes[current_month]

    if (ncol(month_bm_ratios) == sum(is.na(month_bm_ratios))) {
        next
    }
    if (ncol(month_betas) == sum(is.na(month_betas))) {
        next
    }

    # Save regression coefficients and other statistics.
    model <- lm(t(month_returns) ~ 1 + t(month_reversal) + t(month_momentum) + t(month_bm_ratios) + t(month_betas) + t(month_sizes))
    fm_results[i, 1] <- model$coefficients[[1]]
    fm_results[i, 2] <- model$coefficients[[2]]
    fm_results[i, 3] <- model$coefficients[[3]]
    fm_results[i, 4] <- model$coefficients[[4]]
    fm_results[i, 5] <- model$coefficients[[5]]
    fm_results[i, 6] <- model$coefficients[[6]]
    fm_results[i, 7] <- summary(model)$r.squared
    fm_results[i, 8] <- summary(model)$adj.r.squared
    fm_results[i, 9] <- nobs(model)
}

# Compute time series means, standard errors.
final_fm_results <- as.data.frame(matrix(nrow = 2, ncol = 6))
names(final_fm_results) <- c('Intercept', 'Reversal', 'Momentum', 'Value', 'Beta', 'Size')

for (i in 1:ncol(final_fm_results)) {
    model <- lm(na.omit(fm_results[, i]) ~ 1)
    final_fm_results[1, i] <- model$coefficients[[1]]
    final_fm_results[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
}

final_fm_results

In [ ]:
save(
    daily_returns,
    monthly_betas_24m,
    monthly_bm_ratios,
    monthly_ff3_factors,
    monthly_market_caps,
    monthly_momentum,
    monthly_returns,
    monthly_reversal,
    monthly_sizes,
    file = 'seminar_11.RData'
)